# Recognisign Traffic Sign Recognition: Model 2 Detection and Classifcation

**NOTE:Before running the cells below enable the GPU by::::   Click on Runtime -> Change Runtime type -> Hardware Accelarator ->GPU**

## 1. Imports and loading the important files
- ``yolov3_ts_train_last.weights``: The weights of yolo network for detection of traffic sign
- ``yolov3_ts_test.cfg``: The cofiguration file containing the archotecture for yolo model.
- ``varang.h5``: It is the CNN model trained on a lot of traffic signs given in training set

In [ ]:
!gdown --id 1I1rZ-8s7M1_x7xxWob_mU9nT5lFXpGwd  #downloading all files remotely
!gdown --id 1-1Xy6eOcFWmZAqwHmnKeKuYjzt5i8vsm
import zipfile
with zipfile.ZipFile("/content/yolo.zip","r") as zip_ref:
    zip_ref.extractall("/content/")
!cp "/content/yolo/cv2.cpython-36m-x86_64-linux-gnu.so" .
%matplotlib inline
import tensorflow as tf
import cv2
import numpy as np
import pandas as pd 
import time
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import pickle
from keras.models import load_model
from google.colab import files
import os
path_to_weights = '/content/yolo/yolov3_ts_train_last.weights' #load weights for YOLOv3
path_to_cfg = '/content/yolo/yolov3_ts_test.cfg'               #load config file for building YOLO architecture
network = cv2.dnn.readNetFromDarknet(path_to_cfg, path_to_weights) #building model from weights and config file
network.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)           #enabling GPU
network.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)             
layers_all = network.getLayerNames()
layers_names_output = [layers_all[i[0] - 1] for i in network.getUnconnectedOutLayers()]
probability_minimum = 0.3
threshold = 0.2
colours = np.random.randint(0, 255, size=(1, 3), dtype='uint8')
model = tf.keras.models.load_model("/content/my_model_fin.h5")  # loading CNN classification model     
dictionary = {0:"NO ENTRY", 1:"ALL MOTOR VEHICLE PROHIBITED", 2:"HAND CART PROHIBITED",3:"LEFT TURN PROHIBITED",4:"BULLOCK CART AND CART PROHIBITED",5:"HEIGHT LIMIT", 6: "NO PARKING", 7: "ONE WAY SIGN", 8:"TRUCK PROHIBITED",9:"CYCLE PROHIBITED",10:"U TURN PROHIBITED",11: 'WIDTH LIMIT', 12: 'NO STOPPING', 13: 'ONE WAY', 14: 'BULLOCK CART/TONGA PROHIBITED', 15: 'PEDESTRIANS PROHIBITED', 16: 'OVERTAKING PROHIBITED', 17: 'SPEED LIMIT', 18: 'AXLE LOAD LIMIT', 19: 'COMPULSORY AHEAD ONLY', 20: 'VEHICLE PROHIBITED IN BOTH DIRECTIONS', 21: 'RIGHT TURN PROHIBITED', 22: 'HORN PROHIBITED', 23: 'LOAD LIMIT', 24: 'RESTRICTION ENDS SIGN', 25: 'COMPULSORY KEEP LEFT', 26: 'COMPULSORY TURN LEFT', 27: 'COMPULSORY AHEAD OR TURN LEFT',28: 'COMPULSORY TURN RIGHT', 29: 'COMPULSORY AHEAD OR TURN RIGHT', 30: 'COMPULSORY TURN RIGHT AHEAD', 31: 'COMPULSORY CYCLE TRACK', 32: 'COMPULSORY TURN LEFT AHEAD', 33: 'COMPULSORY SOUND HORN', 34: 'STOP', 35: 'GIVE WAY', 36: 'RIGHT HAND CURVE', 37: 'LEFT REVERSE BEND', 38: 'NARROW BRIDGE', 39: 'SCHOOL AHEAD', 40: 'LEFT HAND CURVE',41: 'STEEP ASCENT', 42: 'SLIPPERY ROAD', 43: 'MEN AT WORK', 44: 'RIGHT HAIR PIN BEND', 45:'STEEP DESCENT' , 46: 'LOOSE GRAVEL', 47: 'CATTLE', 48: 'LEFT HAIR PIN BEND', 49: 'NARROW ROAD AHEAD', 50: 'CYCLE CROSSING', 51: 'FALLING ROCKS', 52: 'RIGHT REVERSE BEND', 53: 'ROAD WIDENESS AHEAD', 54: 'PEDESTRIAN CROSSING', 55: 'FERRY', 56: 'CROSS ROAD', 57: 'T INTERSECTION/MAJOR ROAD AHEAD', 58: '200 METERS', 59: 'SIDE ROAD RIGHT', 60: 'ROUND ABOUT', 61: '50-100 METERS', 62: 'SIDE ROAD LEFT', 63: 'STAGGERED INTERSECTION', 64: 'DANGEROUS DIP', 65: 'ROUGH ROAD', 66: 'Y INTERSECTION', 67: 'LENGTH LIMIT' , 68: 'BARRIER AHEAD', 69: 'GAP IN MEDIAN'}

Downloading...
From: https://drive.google.com/uc?id=1I1rZ-8s7M1_x7xxWob_mU9nT5lFXpGwd
To: /content/yolo.zip
489MB [00:05, 86.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-1Xy6eOcFWmZAqwHmnKeKuYjzt5i8vsm
To: /content/my_model_fin.h5
26.6MB [00:00, 73.1MB/s]


##2. Testing on Scenic Images

### Upload Images from local system for testing

In [ ]:
from google.colab import files
uploaded = files.upload()
images=[]
for fn in uploaded.keys():
  img = cv2.imread('/content/'+fn)
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  images.append(img)

### Testing on images

- This is done by passing the image through the YOLO architecture and getting the bouding box. Then the ROI inside BB is sent though the the CNN loaded as ``model``

In [ ]:
for img in images: #looping thorugh images, ignore the jargon
  h, w = img.shape[:2] 
  blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416), swapRB=False, crop=False)
  blob.shape
  network.setInput(blob)
  output_from_network = network.forward(layers_names_output)
  bounding_boxes = []
  confidences = []
  class_numbers = []
  for result in output_from_network:
          for detected_objects in result:
              scores = detected_objects[5:]
              class_current = np.argmax(scores)
              confidence_current = scores[class_current]
              if confidence_current > probability_minimum:   
                  box_current = detected_objects[0:4] * np.array([w, h, w, h])
                  x_center, y_center, box_width, box_height = box_current
                  x_min = int(x_center - (box_width / 2))
                  y_min = int(y_center - (box_height / 2))
                  bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
                  confidences.append(float(confidence_current))
                  class_numbers.append(class_current)
  results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold)
  if len(results) > 0:   
          for i in results.flatten():      
              x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
              box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]
              c_ts = img[y_min:y_min+int(box_height), x_min:x_min+int(box_width), :]
              if c_ts.shape[:1] == (0,) or c_ts.shape[1:2] == (0,):
                  pass
              else:
                  blob_ts = cv2.dnn.blobFromImage(c_ts, 1 / 255.0, size=(32, 32), swapRB=True, crop=False)  
                  blob_ts = blob_ts.transpose(0, 2, 3, 1)
                  prediction = dictionary[np.argmax(model.predict(np.reshape(cv2.cvtColor(cv2.resize(c_ts,(60,60)),cv2.COLOR_RGB2GRAY)/255,(1,60,60,1) )))]  ##
                  colour_box_current = colours[class_numbers[i]].tolist()
                  cv2.rectangle(img, (x_min, y_min), (x_min + box_width, y_min + box_height),colour_box_current, 2)
                  text_box_current = '{}: {:.4f}'.format(prediction,confidences[i])
                  cv2.putText(img, text_box_current, (x_min, y_min - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colour_box_current, 2)
  plt.figure(figsize = (10,10))
  plt.imshow(img)

##3. Testing on Video
- The idea is to loop thorugh the video, and grab a frame and predict upon it.
- On getting a frame the model treats it as an image and performs the same operations as in above cell

### Upload Video

In [ ]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  video = cv2.VideoCapture('/content/'+ fn)

Saving Delhi is Mind-Blowing _ Open & Green - Delhi city tour By Road - YouTube and 8 more pages - Personal - Microsoft​ Edge 2020-12-15 01-33-20.mp4 to Delhi is Mind-Blowing _ Open & Green - Delhi city tour By Road - YouTube and 8 more pages - Personal - Microsoft​ Edge 2020-12-15 01-33-20.mp4


### Testing on Video

In [ ]:
video = cv2.VideoCapture("/content/drive/MyDrive/Recognisign/dataset/Videos/input/inputvadoi.mp4")
writer = None
h, w = None, None
plt.rcParams['figure.figsize'] = (3, 3)
f = 0
t = 0
while True:
    ret, frame = video.read()
    if not ret:
        break
    if w is None or h is None:
        h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),[0,0,0],swapRB=True, crop=False)
    network.setInput(blob)
    start = time.time()
    output_from_network = network.forward(layers_names_output)
    end = time.time()
    f += 1
    t += end - start
    print('Frame number {0} took {1:.5f} seconds'.format(f, end - start))
    bounding_boxes = []
    confidences = []
    class_numbers = []
    for result in output_from_network:
        for detected_objects in result:
            scores = detected_objects[5:]
            class_current = np.argmax(scores)
            confidence_current = scores[class_current]
            if confidence_current > probability_minimum:
                box_current = detected_objects[0:4] * np.array([w, h, w, h])
                x_center, y_center, box_width, box_height = box_current
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))
                bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
                confidences.append(float(confidence_current))
                class_numbers.append(class_current)
    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold)
    if len(results) > 0:
        for i in results.flatten():
            x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
            box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]
            c_ts = frame[y_min:y_min+int(box_height), x_min:x_min+int(box_width), :]        
            if c_ts.shape[:1] == (0,) or c_ts.shape[1:2] == (0,):
                pass
            else:
                blob_ts = cv2.dnn.blobFromImage(c_ts, 1 / 255.0, size=(32, 32), swapRB=True, crop=False)
                blob_ts = blob_ts.transpose(0, 2, 3, 1)
                prediction = dictionary[np.argmax(model.predict(np.reshape(cv2.cvtColor(cv2.resize(c_ts,(60,60)),cv2.COLOR_RGB2GRAY)/255,(1,60,60,1) )))]  
                colour_box_current = colours[class_numbers[i]].tolist()
                cv2.rectangle(frame, (x_min, y_min),
                              (x_min + box_width, y_min + box_height),
                              colour_box_current, 2)
                text_box_current = '{}: {:.4f}'.format(prediction,
                                                       confidences[i])
                cv2.putText(frame, text_box_current, (x_min, y_min - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, colour_box_current, 2)
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')        
        writer = cv2.VideoWriter('/content/drive/MyDrive/Recognisign/dataset/Videos/output/resultsd.mp4', fourcc, 25,
                                 (frame.shape[1], frame.shape[0]), True)
    writer.write(frame)
video.release()
writer.release()
files.download('result.mp4')

Frame number 1 took 3.93233 seconds
Frame number 2 took 0.01589 seconds
Frame number 3 took 0.01586 seconds
Frame number 4 took 0.01588 seconds
Frame number 5 took 0.01586 seconds
Frame number 6 took 0.01585 seconds
Frame number 7 took 0.01584 seconds
Frame number 8 took 0.01586 seconds
Frame number 9 took 0.01586 seconds
Frame number 10 took 0.01587 seconds
Frame number 11 took 0.01586 seconds
Frame number 12 took 0.01588 seconds
Frame number 13 took 0.01588 seconds
Frame number 14 took 0.01588 seconds
Frame number 15 took 0.01588 seconds
Frame number 16 took 0.01587 seconds
Frame number 17 took 0.01585 seconds
Frame number 18 took 0.01587 seconds
Frame number 19 took 0.01587 seconds
Frame number 20 took 0.01590 seconds
Frame number 21 took 0.01586 seconds
Frame number 22 took 0.01588 seconds
Frame number 23 took 0.01587 seconds
Frame number 24 took 0.01589 seconds
Frame number 25 took 0.01587 seconds
Frame number 26 took 0.01590 seconds
Frame number 27 took 0.01586 seconds
Frame numb

FileNotFoundError: ignored